In [1]:
from ase.atoms import Atoms
from ase.io import write
from ase.visualize import view
import e3nn_jax as e3nn
import jax
import jax.numpy as jnp
import jraph
import sys
sys.path.append('..')
import analysis
import datatypes
from input_pipeline_tf import get_datasets
from models import get_first_node_indices, segment_sample
from train import get_predictions

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [2]:
config, best_state, metrics_for_best_state, datasets = analysis.load_from_workdir('../workdirs/mace/interactions=1/l=0/channels=64')

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [3]:
metrics_for_best_state

{'val': {'total_loss': array(8.657066, dtype=float32),
  'focus_loss': array(0.56171024, dtype=float32),
  'atom_type_loss': array(0.86556536, dtype=float32),
  'position_loss': array(7.229792, dtype=float32)},
 'test': {'total_loss': array(8.863261, dtype=float32),
  'focus_loss': array(0.44567674, dtype=float32),
  'atom_type_loss': array(0.8436947, dtype=float32),
  'position_loss': array(7.57389, dtype=float32)}}

In [4]:
cutoff = 5.0
key = jax.random.PRNGKey(0)
epsilon = 1e-4

In [5]:
example_graph = next(datasets["test"].as_numpy_iterator())
frag = datatypes.Fragment.from_graphstuple(example_graph)
frag = jax.tree_map(jnp.asarray, frag)
num_nodes = frag.nodes.positions.shape[0]
num_graphs = frag.n_node.shape[0]

In [6]:
frag_unpadded = jraph.unpad_with_graphs(frag)
molecules = jraph.unbatch(frag_unpadded)

In [7]:
mol = molecules[2]

In [8]:
preds = get_predictions(best_state, mol, key)
true_focus_indices = get_first_node_indices(mol)

# add stop
focus_probs = jax.nn.softmax(jnp.concatenate([preds.focus_logits, jnp.array([0])]))
# jnp.set_printoptions(precision=3, suppress=True)

In [13]:
atomic_numbers = jnp.array([1, 6, 7, 8, 9])
numbers_to_symbols = {1: 'H', 6: 'C', 7: 'N', 8: 'O', 9: 'F'}
elements = list(numbers_to_symbols.values())

# covalent bond radii, in angstroms
element_radii = [0.32, 0.75, 0.71, 0.63, 0.64]

def get_numbers(species: jnp.ndarray):
    numbers = []
    for i in species:
        numbers.append(atomic_numbers[i])
    return jnp.array(numbers)

In [12]:
pred_species = preds.target_species.tolist()[0]

In [11]:
preds.position_coeffs

1x0e
[[[-4.120972  ]
  [-4.092948  ]
  [-5.1138535 ]
  [-5.158257  ]
  [-4.5202165 ]
  [-3.386344  ]
  [-4.229885  ]
  [-3.3213027 ]
  [-2.0735316 ]
  [-1.7883668 ]
  [-0.5897599 ]
  [ 0.4420494 ]
  [ 1.3838742 ]
  [ 3.2845714 ]
  [ 4.5754986 ]
  [ 5.726662  ]
  [ 6.4547367 ]
  [ 6.7055197 ]
  [ 6.6437073 ]
  [ 6.1172595 ]
  [ 5.2017245 ]
  [ 4.1756997 ]
  [ 2.4395177 ]
  [ 0.05779187]
  [-2.2834876 ]
  [-2.4535067 ]
  [-3.1805484 ]
  [-2.590728  ]
  [-1.7091949 ]
  [-2.7569616 ]
  [-1.4706519 ]
  [-1.2866036 ]
  [-1.5887159 ]
  [-2.7202535 ]
  [-3.4202318 ]
  [-2.2981281 ]
  [-4.0578175 ]
  [-1.5514712 ]
  [-1.6524652 ]
  [-1.9525069 ]
  [-2.5296748 ]
  [-3.1790006 ]
  [-3.811808  ]
  [-2.9042213 ]
  [-2.3503883 ]
  [-2.6280155 ]
  [-4.322109  ]
  [-3.6532114 ]
  [-3.550451  ]
  [-4.1777525 ]
  [-4.6460443 ]
  [-5.419183  ]
  [-6.019831  ]
  [-4.622957  ]
  [-3.7174263 ]
  [-5.173094  ]
  [-3.699925  ]
  [-3.9911768 ]
  [-3.7409046 ]
  [-5.400814  ]
  [-4.7032943 ]
  [-4.132673  ]
  [

In [84]:
mol_atoms = Atoms(positions=mol.nodes.positions.tolist(), numbers=get_numbers(mol.nodes.species.tolist()))
v = view(mol_atoms, viewer='ngl')

# add focus probability highlights for each atom
for i in range(mol.n_node[0]):
    focus_prob = focus_probs.tolist()[i]
    v.view.shape.add_sphere(
        mol.nodes.positions[i].tolist(),
        [1, 0.85, 0],
        element_radii[mol.nodes.species.tolist()[i]] * 0.6,
        f"atom {i}: focus probability {focus_prob:.3f}",
    )
    v.view.update_representation(component=i+1, opacity=focus_prob)
v

# add the next atom we're adding to this molecule, predicted specie + highlight
v.view.shape.add_sphere(
    mol.nodes.positions[i].tolist(),
    [1, 0, 1],
    element_radii[mol.nodes.species.tolist()[i]],
    f"predicted atom: {pred_species}",
)